# Mapping the human body

The Human BioMolecular Atlas Program is a major endeavour working to catalyze the development of a framework for mapping the human body at single cell resolution.One component of this overarching goal is to identify medically relevant functional tissue units (FTUs) within whole slide microscopy images of human tissues. Once these FTUs are detected, information on size, shape, variability in number and location within the tissue samples can be used to help build a spatially accurate and semantically explicit model of the human body.



In [17]:
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/yCh4XnD7rEE" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')



# Framing the problem

Our best estimates show there are over 7 billion people on the planet and 300 billion stars in the Milky Way galaxy. By comparison, the adult human body contains 37 trillion cells. To determine the function and relationship among these cells is a monumental undertaking. Many areas of human health would be impacted if we better understand cellular activity. A problem with this much data is a great match for the Kaggle community.

Just as the Human Genome Project mapped the entirety of human DNA, the Human BioMolecular Atlas Program (HuBMAP) is a major endeavor. Sponsored by the National Institutes of Health (NIH), HuBMAP is working to catalyze the development of a framework for mapping the human body at a level of glomeruli functional tissue units for the first time in history. Hoping to become one of the world’s largest collaborative biological projects, HuBMAP aims to be an open map of the human body at the cellular level.

This competition, “Hacking the Kidney," starts by mapping the human kidney at single cell resolution.

Your challenge is to detect functional tissue units (FTUs) across different tissue preparation pipelines. An FTU is defined as a “three-dimensional block of cells centered around a capillary, such that each cell in this block is within diffusion distance from any other cell in the same block” (de Bono, 2013). The goal of this competition is the implementation of a successful and robust glomeruli FTU detector.

You will also have the opportunity to present your findings to a panel of judges for additional consideration. Successful submissions will construct the tools, resources, and cell atlases needed to determine how the relationships between cells can affect the health of an individual.

Advancements in HuBMAP will accelerate the world’s understanding of the relationships between cell and tissue organization and function and human health. These datasets and insights can be used by researchers in cell and tissue anatomy, pharmaceutical companies to develop therapies, or even parents to show their children the magnitude of the human body.


# Understanding the objectif

* From kidney images we need to detect and map FTUs.
* This compeition starts by mapping the human kidney at single cell resolution. We will detect the functional tissue units(FTU) accross different tissue preparation pipelines.By this we will implement a successful and robust glomeruli FTU detector.
* We also have the opportunity to present our findings to a panel of judges of this competition for additional consideration

# What is a FTU ?

An FTU is defined as a “three-dimensional block of cells centered around a capillary, such that each cell in this block is within diffusion distance from any other cell in the same block”. Bernard de Bono and his team coined the term “functional tissue unit” in 2013 as: “...a three-dimensional block of cells centered around a capillary, such that each cell in this block is within diffusion distance from any other cell in the same block” (de Bono, 2013) One example of an FTU is the glomerulus found in the outer layer of kidney tissue known as the cortex, which in humans has an area of about 800 mm2 and average depth of about 9 mm (Mounier-Vehier, 2002).

# Metric : Dice coefficient 

The evaluation metric of this competition is Dice Coefficient. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. The formula is given by:

$$\frac{2* | {X}\bigcap{Y} | }{ |X| + |Y|}$$

where X is the predicted set of pixels and Y is the ground truth.

![dice coefficient](https://miro.medium.com/max/858/1*yUd5ckecHjWZf6hGrdlwzA.png)

Dice coefficient, is a statistical tool which measures the similarity between two sets of data. This index has become arguably the most broadly used tool in the validation of image segmentation algorithms created with AI, but it is a much more general concept which can be applied sets of data for a variety of applications including NLP.



# Previous research

## Automatic glomerular identification and quantification of histological phenotypes using image analysis and machine learning (Sheehan and Korstanje 2018)

Current methods of scoring histological kidney samples, specifically glomeruli, do not allow for collection of quantitative data in a high-throughput and consistent manner. Neither untrained individuals nor computers are presently capable of identifying glomerular features, so expert pathologists must do the identification and score using a categorical matrix, complicating statistical analysis. Critical information regarding overall health and physiology is encoded in these samples. Rapid comprehensive histological scoring could be used, in combination with other physiological measures, to significantly advance renal research. Therefore, we used machine learning to develop a high-throughput method to automatically identify and collect quantitative data from glomeruli. Our method requires minimal human interaction between steps and provides quantifiable data independent of user bias. The method uses free existing software and is usable without extensive image analysis training. Validation of the classifier and feature scores in mice is highlighted in this work and shows the power of applying this method in murine research. Preliminary results indicate that the method can be applied to data sets from different species after training on relevant data, allowing for fast glomerular identification and quantitative measurements of glomerular features. Validation of the classifier and feature scores are highlighted in this work and show the power of applying this method. The resulting data are free from user bias. Continuous data, such that statistical analysis can be performed, allows for more precise and comprehensive interrogation of samples. These data can then be combined with other physiological data to broaden our overall understanding of renal function.

## Glomerulus Classification and Detection Based on Convolutional Neural Networks (Gallego et al. 2018)

Glomerulus classification and detection in kidney tissue segments are key processes in nephropathology used for the correct diagnosis of the diseases. In this paper, we deal with the challenge of automating Glomerulus classification and detection from digitized kidney slide segments using a deep learning framework. The proposed method applies Convolutional Neural Networks (CNNs) between two classes: Glomerulus and Non-Glomerulus, to detect the image segments belonging to Glomerulus regions. We configure the CNN with the public pre-trained AlexNet model and adapt it to our system by learning from Glomerulus and Non-Glomerulus regions extracted from training slides. Once the model is trained, labeling is performed by applying the CNN classification to the image blocks under analysis. The results of the method indicate that this technique is suitable for correct Glomerulus detection in Whole Slide Images (WSI), showing robustness while reducing false positive and false negative detections.

## Region-Based Convolutional Neural Nets for Localization of Glomeruli in Trichrome-Stained Whole Kidney Sections (Bukowy et al. 2018)

Background Histologic examination of fixed renal tissue is widely used to assess morphology and the progression of disease. Commonly reported metrics include glomerular number and injury. However, characterization of renal histology is a time-consuming and user-dependent process. To accelerate and improve the process, we have developed a glomerular localization pipeline for trichrome-stained kidney sections using a machine learning image classification algorithm.

Methods We prepared 4-μm slices of kidneys from rats of various genetic backgrounds that were subjected to different experimental protocols and mounted the slices on glass slides. All sections used in this analysis were trichrome stained and imaged in bright field at a minimum resolution of 0.92 μm per pixel. The training and test datasets for the algorithm comprised 74 and 13 whole renal sections, respectively, totaling over 28,000 glomeruli manually localized. Additionally, because this localizer will be ultimately used for automated assessment of glomerular injury, we assessed bias of the localizer for preferentially identifying healthy or damaged glomeruli.

Results Localizer performance achieved an average precision and recall of 96.94% and 96.79%, respectively, on whole kidney sections without evidence of bias for or against glomerular injury or the need for manual preprocessing.

Conclusions This study presents a novel and robust application of convolutional neural nets for the localization of glomeruli in healthy and damaged trichrome-stained whole-renal section mounts and lays the groundwork for automated glomerular injury scoring.

## Segmentation of Glomeruli Within Trichrome Images Using Deep Learning (Kannan et al. 2019)

Introduction The number of glomeruli and glomerulosclerosis evaluated on kidney biopsy slides constitute standard components of a renal pathology report. Prevailing methods for glomerular assessment remain manual, labor intensive, and nonstandardized. We developed a deep learning framework to accurately identify and segment glomeruli from digitized images of human kidney biopsies. Methods Trichrome-stained images (n = 275) from renal biopsies of 171 patients with chronic kidney disease treated at the Boston Medical Center from 2009 to 2012 were analyzed. A sliding window operation was defined to crop each original image to smaller images. Each cropped image was then evaluated by at least 3 experts into 3 categories: (i) no glomerulus, (ii) normal or partially sclerosed (NPS) glomerulus, and (iii) globally sclerosed (GS) glomerulus. This led to identification of 751 unique images representing nonglomerular regions, 611 images with NPS glomeruli, and 134 images with GS glomeruli. A convolutional neural network (CNN) was trained with cropped images as inputs and corresponding labels as output. Using this model, an image processing routine was developed to scan the test images to segment the GS glomeruli. Results The CNN model was able to accurately discriminate nonglomerular images from NPS and GS images (performance on test data: Accuracy: 92.67% ± 2.02% and Kappa: 0.8681 ± 0.0392). The segmentation model that was based on the CNN multilabel classifier accurately marked the GS glomeruli on the test data (Matthews correlation coefficient = 0.628). Conclusion This work demonstrates the power of deep learning for assessing complex histologic structures from digitized human kidney biopsies.

# EDA

In [7]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import cv2
import tifffile

In [9]:
BASE_PATH = "../input/hubmap-kidney-segmentation/"
TRAIN_PATH = os.path.join(BASE_PATH, "train")

print(os.listdir(BASE_PATH))

['sample_submission.csv', 'HuBMAP-20-dataset_information.csv', 'train.csv', 'test', 'train']


In [13]:
df_train = pd.read_csv(
    os.path.join(BASE_PATH, "train.csv")
)
df_train

,id,encoding
0,2f6ecfcdf,296084587 4 296115835 6 296115859 14 296147109...
1,aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...
2,cb2d976f4,78144363 5 78179297 15 78214231 25 78249165 35...
3,0486052bb,101676003 6 101701785 8 101727568 9 101753351 ...
4,e79de561c,7464094 14 7480273 41 7496453 67 7512632 82 75...
5,095bf7a1f,113430380 22 113468538 67 113506697 111 113544...
6,54f2eec69,124601765 36 124632133 109 124662536 147 12469...
7,1e2425f28,49453112 7 49479881 22 49506657 31 49533433 40...


In [14]:
df_info = pd.read_csv(
    os.path.join(BASE_PATH, "HuBMAP-20-dataset_information.csv")
)
df_info.sample(3)

,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla
1,54f2eec69.tiff,22240,30440,54f2eec69-anatomical-structure.json,54f2eec69.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
11,0486052bb.tiff,34937,25784,0486052bb-anatomical-structure.json,0486052bb.json,67177,White,Not Hispanic or Latino,Male,31,106.1,180.3,32.6,Right,80,20
8,b9a3865fc.tiff,40429,31295,b9a3865fc-anatomical-structure.json,b9a3865fc.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45


In [ ]:
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [
        np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape).T

def read_image(image_id, scale=None, verbose=1):
    image = tifffile.imread(
        os.path.join(BASE_PATH, f"train/{image_id}.tiff")
    )
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    mask = rle2mask(
        df_train[df_train["id"] == image_id]["encoding"].values[0], 
        (image.shape[1], image.shape[0])
    )
    
    if verbose:
        print(f"[{image_id}] Image shape: {image.shape}")
        print(f"[{image_id}] Mask shape: {mask.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        mask = cv2.resize(mask, new_size)
        
        if verbose:
            print(f"[{image_id}] Resized Image shape: {image.shape}")
            print(f"[{image_id}] Resized Mask shape: {mask.shape}")
        
    return image, mask

def read_test_image(image_id, scale=None, verbose=1):
    image = tifffile.imread(
        os.path.join(BASE_PATH, f"test/{image_id}.tiff")
    )
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    if verbose:
        print(f"[{image_id}] Image shape: {image.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        
        if verbose:
            print(f"[{image_id}] Resized Image shape: {image.shape}")
        
    return image

def plot_image_and_mask(image, mask, image_id):
    plt.figure(figsize=(16, 10))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title(f"Image {image_id}", fontsize=18)
    
    plt.subplot(1, 3, 2)
    plt.imshow(image)
    plt.imshow(mask, cmap="hot", alpha=0.5)
    plt.title(f"Image {image_id} + mask", fontsize=18)    
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap="hot")
    plt.title(f"Mask", fontsize=18)    
    
    plt.show()
    
    
def plot_grid_image_with_mask(image, mask):
    plt.figure(figsize=(16, 16))
    
    w_len = image.shape[0]
    h_len = image.shape[1]
    
    min_len = min(w_len, h_len)
    w_start = (w_len - min_len) // 2
    h_start = (h_len - min_len) // 2
    
    plt.imshow(image[w_start : w_start + min_len, h_start : h_start + min_len])
    plt.imshow(
        mask[w_start : w_start + min_len, h_start : h_start + min_len], cmap="hot", alpha=0.5,
    )
    plt.axis("off")
            
    plt.show()
    
def plot_slice_image_and_mask(image, mask, start_h, end_h, start_w, end_w):
    plt.figure(figsize=(16, 5))
    
    sub_image = image[start_h:end_h, start_w:end_w, :]
    sub_mask = mask[start_h:end_h, start_w:end_w]
    
    plt.subplot(1, 3, 1)
    plt.imshow(sub_image)
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(sub_image)
    plt.imshow(sub_mask, cmap="hot", alpha=0.5)
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(sub_mask, cmap="hot")
    plt.axis("off")
    
    plt.show()



In [ ]:
small_ids = [
    "0486052bb", "095bf7a1f", "1e2425f28", "2f6ecfcdf",
    "54f2eec69", "aaa6a05cc", "cb2d976f4", "e79de561c",
]
small_images = []
small_masks = []

for small_id in small_ids:
    tmp_image, tmp_mask = read_image(small_id, scale=20, verbose=0)
    small_images.append(tmp_image)
    small_masks.append(tmp_mask)

In [ ]:
plt.figure(figsize=(16, 16))
for ind, (tmp_id, tmp_image) in enumerate(zip(small_ids, small_images)):
    plt.subplot(3, 3, ind + 1)
    plt.imshow(tmp_image)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(16, 16))
for ind, (tmp_id, tmp_image, tmp_mask) in enumerate(zip(small_ids, small_images, small_masks)):
    plt.subplot(3, 3, ind + 1)
    plt.imshow(tmp_image)
    plt.imshow(tmp_mask, cmap="hot", alpha=0.5)
    plt.axis("off")

# Metadata analysis

In [10]:
pd.read_json(
    os.path.join(BASE_PATH, "train/0486052bb-anatomical-structure.json")
)

,type,id,geometry,properties
0,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[20606, 4...","{'classification': {'name': 'Medulla', 'colorR..."
1,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[30717, 1...","{'classification': {'name': 'Cortex', 'colorRG..."


In [11]:
pd.read_json(
    os.path.join(BASE_PATH, "train/0486052bb.json")
)

,type,id,geometry,properties
0,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[4747, 96...","{'classification': {'name': 'glomerulus', 'col..."
1,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[6631, 67...","{'classification': {'name': 'glomerulus', 'col..."
2,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[6287, 10...","{'classification': {'name': 'glomerulus', 'col..."
3,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[6579, 13...","{'classification': {'name': 'glomerulus', 'col..."
4,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[8779, 11...","{'classification': {'name': 'glomerulus', 'col..."
...,...,...,...,...
121,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[23073, 2...","{'classification': {'name': 'glomerulus', 'col..."
122,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[20891, 5...","{'classification': {'name': 'glomerulus', 'col..."
123,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[15277, 2...","{'classification': {'name': 'glomerulus', 'col..."
124,Feature,PathAnnotationObject,"{'type': 'Polygon', 'coordinates': [[[7172, 85...","{'classification': {'name': 'glomerulus', 'col..."


In [15]:
df_info["split"] = "test"
df_info.loc[df_info["image_file"].isin(os.listdir(os.path.join(BASE_PATH, "train"))), "split"] = "train"

In [16]:
df_info["area"] = df_info["width_pixels"] * df_info["height_pixels"]
df_info.head()

,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla,split,area
0,afa5e8098.tiff,43780,36800,afa5e8098-anatomical-structure.json,afa5e8098.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45,test,1611104000
1,54f2eec69.tiff,22240,30440,54f2eec69-anatomical-structure.json,54f2eec69.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25,train,676985600
2,1e2425f28.tiff,32220,26780,1e2425f28-anatomical-structure.json,1e2425f28.json,63921,White,Not Hispanic or Latino,Male,48,131.5,193.0,35.3,Right,65,35,train,862851600
3,e79de561c.tiff,27020,16180,e79de561c-anatomical-structure.json,e79de561c.json,67026,Black or African American,Not Hispanic or Latino,Male,53,73.0,166.0,26.5,Left,55,45,train,437183600
4,c68fe75ea.tiff,19780,26840,c68fe75ea-anatomical-structure.json,c68fe75ea.json,67112,White,Not Hispanic or Latino,Male,56,91.2,167.6,32.5,Left,80,20,test,530895200


# Research on data

## The kidney nephron

Each of your kidneys is made up of about a million filtering units called nephrons. Each nephron includes a filter, called the glomerulus, and a tubule. The nephrons work through a two-step process: the glomerulus filters your blood, and the tubule returns needed substances to your blood and removes wastes.

![nephron](https://www.niddk.nih.gov/-/media/Images/Health-Information/Kidney/Nephron_Extraction_330x380.png)

Each nephron has a glomerulus to filter your blood and a tubule that returns needed substances to your blood and pulls out additional wastes. Wastes and extra water become urine.

## The glomerulus filters the blood

As blood flows into each nephron, it enters a cluster of tiny blood vessels—the glomerulus. The thin walls of the glomerulus allow smaller molecules, wastes, and fluid—mostly water—to pass into the tubule. Larger molecules, such as proteins and blood cells, stay in the blood vessel.

![glomerulus](https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Renal_corpuscle-en.svg/400px-Renal_corpuscle-en.svg.png)

## How the blood flow through the kidney ?

Blood flows into your kidney through the renal artery. This large blood vessel branches into smaller and smaller blood vessels until the blood reaches the nephrons. In the nephron, your blood is filtered by the tiny blood vessels of the glomeruli and then flows out of your kidney through the renal vein.

Your blood circulates through your kidneys many times a day. In a single day, your kidneys filter about 150 quarts of blood. Most of the water and other substances that filter through your glomeruli are returned to your blood by the tubules. Only 1 to 2 quarts become urine.

![bloodflow](https://www.niddk.nih.gov/-/media/Images/Health-Information/Kidney/Kidney_1_330x315.png)

Blood flows into your kidneys through the renal artery and exits through the renal vein. Your ureter carries urine from the kidney to your bladder.

## The function of filtration

The main function of the glomerulus is to filter plasma to produce glomerular filtrate, which passes down the length of the nephron tubule to form urine. The rate at which the glomerulus produces filtrate from plasma (the glomerular filtration rate) is much higher than in systemic capillaries because of the particular anatomical characteristics of the glomerulus. Unlike systemic capillaries, which receive blood from high-resistance arterioles and drain to low-resistance venules, glomerular capillaries are connected in both ends to high-resistance arterioles: the afferent arteriole, and the efferent arteriole. This arrangement of two arterioles in series determines the high hydrostatic pressure on glomerular capillaries, which is one of the forces that favour filtration to the Bowman's capsule.

If a substance has passed through the glomerular capillary endothelial cells, glomerular basement membrane, and podocytes, then it enters the lumen of the tubule and is known as glomerular filtrate. Otherwise, it exits the glomerulus through the efferent arteriole and continues circulation as discussed below and as shown on the picture.

## Zoom on glomerulus

![](https://www.auanet.org/images/Education/pathology/normal-histology/renal_corpuscle-figureA_Big.jpg)

![](https://www.auanet.org/images/Education/pathology/normal-histology/renal_corpuscle-figureB_Big.jpg)


## Damage on glomerulus

Damage to the glomerulus by disease can allow passage through the glomerular filtration barrier of red blood cells, white blood cells, platelets, and blood proteins such as albumin and globulin. Underlying causes for glomerular injury can be inflammatory, toxic or metabolic. These can be seen in the urine (urinalysis) on microscopic and chemical (dipstick) examination. Examples are diabetic kidney disease, glomerulonephritis, and IgA nephropathy.

Due to the connection between the glomerulus and the glomerular filtration rate, the glomerular filtration rate is of clinical significance when suspecting a kidney disease, or when following up a case with known kidney disease, or when risking a development of renal damage such as beginning medications with known nephrotoxicity.

## Detection methods

Glomerulus classification and detection in kidney tissue segments are key processes in nephropathology used for the correct diagnosis of the diseases. Technics to detect glomerulus involve Convolutional Neural Networks (CNN) between the two classes : Glomerulus and Non-Glomerulus, to detect the image segments belonging to Glomerulus regions.

* Size and shape: In a healthy kidney before sectioning, Glomeruli present a spherical shape with fixed size (diameter ranges between 350 and 100 μm), but its aspect can change due to the presence of medical diseases. For instance: Glomeruli can present a swell aspect under hypertension or diabetes conditions. After sectioning, the presence of pathologies affect the appearance inside the VS. Besides, the different Glomeruli sizes observed could vary depending on where the cross-section was taken with respect to each Glomerulus sphere.

* Color: In some configurations, the use of PAS (Periodic Acid Schiff) stain in tissue sections can give a purple-magenta color to the slides. The amount of stain present in each slide will determine the color intensity of the segments under analysis. Since this process is not perfect, each slide can present different intensities. Moreover, the presence of medical diseases can vary the amount of stain present in the Glomeruli under study.

Glomerulus vs Non-glomerulus

![](https://www.mdpi.com/jimaging/jimaging-04-00020/article_deploy/html/images/jimaging-04-00020-g002.png)


## Conclusions on glomerulus

* Glomerulus are tiny blood vessels that serve as a barrier for the filtration of the plasma.
* Glomerular capillaries are connected in both ends to high-resistance arterioles.
* Damage on glomerulus can be seen in urine tests, microscopic and chemical examinations.
* Glomerular filtration rate is of clinical significance.



# General Analysis

# Assumptions

# Preprocessing

## Image data

* Resize needs
* Tensor transformation
* Normalization

## Metadata

In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image

In [5]:
def image_preprocessing(image_path, resize_amount, normalization):
    
    input_image = Image.open(image_path)
    
    preprocess = transforms.Compose([
      transforms.Resize(resize_amount),
      transforms.ToTensor(),
      transforms.Normalize(normalization),
    ])
    
    return preprocess(input_image)

# Models analysis

# Proposed architecture

# Models training

# Hyperparameters

# Evaluation

# Submission

# Github

* https://github.com/MasMedIm/HuBMAP_Kidney.git

# References

* https://atcold.github.io/pytorch-Deep-Learning/en/week03/03-1/
* https://atcold.github.io/pytorch-Deep-Learning/en/week04/04-1/
* https://www.youtube.com/watch?v=AjtX1N_VT9E
* https://www.kaggle.com/ihelon/hubmap-exploratory-data-analysis
* https://www.niddk.nih.gov/health-information/kidney-disease/kidneys-how-they-work#:~:text=Each%20of%20your%20kidneys%20is,your%20blood%20and%20removes%20wastes.
* https://en.wikipedia.org/wiki/Glomerulus_(kidney)
* https://www.mdpi.com/2313-433X/4/1/20/htm


# License 

[Attribution 4.0 International (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/)
